## **3장 표본추출, 데이터 분할, 교차검증**

### **3-1. 표본추출**

#### 단순랜덤추출법

In [2]:
# Scikit learn의 내장 데이터 iris 불러오기
from sklearn.datasets import load_iris
import numpy as np
from pandas import DataFrame

data = load_iris()
iris_cols = list(data['feature_names']) + ['target']
iris = DataFrame(np.c_[data['data'], data['target']], columns = [col.replace(" (cm)", "") for col in iris_cols])
print(iris.head(5))

   sepal length  sepal width  petal length  petal width  target
0           5.1          3.5           1.4          0.2     0.0
1           4.9          3.0           1.4          0.2     0.0
2           4.7          3.2           1.3          0.2     0.0
3           4.6          3.1           1.5          0.2     0.0
4           5.0          3.6           1.4          0.2     0.0


In [3]:
iris.sample(n=3, replace=False)

,sepal length,sepal width,petal length,petal width,target
0,5.1,3.5,1.4,0.2,0.0
100,6.3,3.3,6.0,2.5,2.0
143,6.8,3.2,5.9,2.3,2.0


In [6]:
iris.sample(frac=0.3).head(3)

,sepal length,sepal width,petal length,petal width,target
127,6.1,3.0,4.9,1.8,2.0
94,5.6,2.7,4.2,1.3,1.0
80,5.5,2.4,3.8,1.1,1.0


In [18]:
iris.sample(n=3, axis=1).head(3)

,petal width,target,sepal width
0,0.2,0.0,3.5
1,0.2,0.0,3.0
2,0.2,0.0,3.2


In [34]:
import random
import numpy as np

data_list = [1,2,3,4,5,'a','b','c']
print(random.sample(data_list,4))
print(np.random.choice(data_list,4))

print(np.random.randint(0,10,3))
print(np.random.rand(2,2))

[3, 2, 1, 'b']
['5' '1' 'c' '3']
[4 5 9]
[[0.58358362 0.35019025]
 [0.09830251 0.31860122]]


#### 계통추출법

In [47]:
import pandas as pd
data, n = iris, 8
N = len(data)
K = N//n  #구간 내 샘플 수
index = data.loc[:K].sample(1).index  #첫 구간에서 임의로 선택한 샘플 인덱스
sys_df = DataFrame()
while len(sys_df) < n:
    sys_df = pd.concat ([sys_df, data.loc[index,:]])
    index += K

print(f'N: {N}')
print(f'n: {n}')
print(f'K: {K}')
print(sys_df)

N: 150
n: 8
K: 18
     sepal length  sepal width  petal length  petal width  target
11            4.8          3.4           1.6          0.2     0.0
29            4.7          3.2           1.6          0.2     0.0
47            4.6          3.2           1.4          0.2     0.0
65            6.7          3.1           4.4          1.4     1.0
83            6.0          2.7           5.1          1.6     1.0
101           5.8          2.7           5.1          1.9     2.0
119           6.0          2.2           5.0          1.5     2.0
137           6.4          3.1           5.5          1.8     2.0


#### 집락 추출법
군집별로 랜덤추출법 수행 / 지역 & 다단계표본추출 / 층화추출법에서 층을 집락으로 대체

#### 층화추출법

In [49]:
# target을 층 혹은 집락이라고 가정!
# 원본 데이터의 분포 확인
from pandas import DataFrame, concat
print(iris['target'].value_counts())

target
0.0    50
1.0    50
2.0    50
Name: count, dtype: int64


In [50]:
# 데이터, 층/집락 정보를 가진 컬럼명, 추출표본 개수
data, stratum, sampling_no = iris, 'target', 9

# 비례층화추출법: 원본 데이터의 비율대로 추출
levels = data[stratum].unique()
total = data[stratum].value_counts().sum()
prop_val = data[stratum].value_counts()/total
no = prop_val * sampling_no
result = DataFrame()
for level in levels:
    temp_df = data[data[stratum]==level].sample(int(no[level]))
    result = concat([result, temp_df])
print(result)

     sepal length  sepal width  petal length  petal width  target
21            5.1          3.7           1.5          0.4     0.0
8             4.4          2.9           1.4          0.2     0.0
5             5.4          3.9           1.7          0.4     0.0
76            6.8          2.8           4.8          1.4     1.0
79            5.7          2.6           3.5          1.0     1.0
62            6.0          2.2           4.0          1.0     1.0
142           5.8          2.7           5.1          1.9     2.0
134           6.1          2.6           5.6          1.4     2.0
110           6.5          3.2           5.1          2.0     2.0


In [59]:
# 불비례층화추출법: 임의로 정한 특정 비율대로 샘플링
# 데이터, 층/집락 정보를 가진 컬럼명, 추출표본 개수, 각 층/집락의 비율
data, stratum, sampling_no, proportion = iris, 'target', 10, {0:0.2, 1:0.5, 2:0.3}

levels = list(proportion.keys())
prop_val = np.array(list(proportion.values()))
no = prop_val * sampling_no
result = DataFrame()
for level in levels:
    temp_df = data[data[stratum]==level].sample(int(no[level]))
    result = concat([result, temp_df])
print(result)

     sepal length  sepal width  petal length  petal width  target
33            5.5          4.2           1.4          0.2     0.0
43            5.0          3.5           1.6          0.6     0.0
64            5.6          2.9           3.6          1.3     1.0
58            6.6          2.9           4.6          1.3     1.0
92            5.8          2.6           4.0          1.2     1.0
85            6.0          3.4           4.5          1.6     1.0
53            5.5          2.3           4.0          1.3     1.0
143           6.8          3.2           5.9          2.3     2.0
102           7.1          3.0           5.9          2.1     2.0
126           6.2          2.8           4.8          1.8     2.0


### **3-2. 데이터 분할**

#### 일반적 데이터 분할 및 홀드아웃 방법

In [61]:
from sklearn.model_selection import train_test_split
X = iris.drop('target', axis=1)
y = iris.filter(['target'])

# 일반적 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
print("X_train(y_train): %d(%d), X_test(y_test): %d(%d)"%(len(X_train), len(y_train), len(X_test), len(y_test)))
print("X_train의 비율: %0.2f, X_test의 비율: %0.2f" %(len(X_train)/len(X), len(X_test)/len(X)), "\n")

# 홀드아웃방법
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)
print("X_train(y_train): %d(%d), X_test(y_test): %d(%d)"%(len(X_train), len(y_train), len(X_test), len(y_test)))
print("X_train의 비율: %0.2f, X_test의 비율: %0.2f" %(len(X_train)/len(X), len(X_test)/len(X)))

X_train(y_train): 105(105), X_test(y_test): 45(45)
X_train의 비율: 0.70, X_test의 비율: 0.30 

X_train(y_train): 75(75), X_test(y_test): 75(75)
X_train의 비율: 0.50, X_test의 비율: 0.50


#### Shuffle Split

In [63]:
from sklearn.model_selection import ShuffleSplit
from collections import Counter
ss = ShuffleSplit(test_size=0.5, train_size=0.5, n_splits=4)
for i, (train_index, test_index) in enumerate(ss.split(X)):
    print(f"Sample {i} ==> train_index: {train_index[:3]}, test_index: {test_index[:3]}")
    X_train, X_test, y_train, y_test = X.iloc[train_index, :], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    print("\tX_train의 비율: %0.2f, X_test의 비율: %0.2f" %(len(X_train)/len(X), len(X_test)/len(X)))
    print("\ty_train의 타겟 구성:", Counter(y_train['target']))
    print("\ty_test의 타겟 구성:", Counter(y_test['target']), "\n")

Sample 0 ==> train_index: [ 71  42 107], test_index: [145  19  92]
	X_train의 비율: 0.50, X_test의 비율: 0.50
	y_train의 타겟 구성: Counter({0.0: 28, 2.0: 27, 1.0: 20})
	y_test의 타겟 구성: Counter({1.0: 30, 2.0: 23, 0.0: 22}) 

Sample 1 ==> train_index: [ 30 115 110], test_index: [ 85 100  61]
	X_train의 비율: 0.50, X_test의 비율: 0.50
	y_train의 타겟 구성: Counter({0.0: 26, 2.0: 25, 1.0: 24})
	y_test의 타겟 구성: Counter({1.0: 26, 2.0: 25, 0.0: 24}) 

Sample 2 ==> train_index: [147 130 104], test_index: [ 30 146  17]
	X_train의 비율: 0.50, X_test의 비율: 0.50
	y_train의 타겟 구성: Counter({1.0: 28, 2.0: 24, 0.0: 23})
	y_test의 타겟 구성: Counter({0.0: 27, 2.0: 26, 1.0: 22}) 

Sample 3 ==> train_index: [ 58  23 142], test_index: [  3  22 118]
	X_train의 비율: 0.50, X_test의 비율: 0.50
	y_train의 타겟 구성: Counter({1.0: 27, 2.0: 26, 0.0: 22})
	y_test의 타겟 구성: Counter({0.0: 28, 2.0: 24, 1.0: 23}) 



#### K-fold 분할

In [66]:
from sklearn.model_selection import KFold
from collections import Counter
#n_splits=fold 개수, shuffle=데이터 분할 전 shuffle 여부
kf = KFold(n_splits=4, shuffle=False) 
for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Sample {i} ==> train_index: {train_index[:3]}, test_index: {test_index[:3]}")
    X_train, X_test, y_train, y_test = X.iloc[train_index, :], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    print("\tX_train의 비율: %0.2f, X_test의 비율: %0.2f" %(len(X_train)/len(X), len(X_test)/len(X)))
    print("\ty_train의 타겟 구성:", Counter(y_train['target']))
    print("\ty_test의 타겟 구성:", Counter(y_test['target']), "\n")

Sample 0 ==> train_index: [38 39 40], test_index: [0 1 2]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({1.0: 50, 2.0: 50, 0.0: 12})
	y_test의 타겟 구성: Counter({0.0: 38}) 

Sample 1 ==> train_index: [0 1 2], test_index: [38 39 40]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({2.0: 50, 0.0: 38, 1.0: 24})
	y_test의 타겟 구성: Counter({1.0: 26, 0.0: 12}) 

Sample 2 ==> train_index: [0 1 2], test_index: [76 77 78]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({0.0: 50, 2.0: 37, 1.0: 26})
	y_test의 타겟 구성: Counter({1.0: 24, 2.0: 13}) 

Sample 3 ==> train_index: [0 1 2], test_index: [113 114 115]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({0.0: 50, 1.0: 50, 2.0: 13})
	y_test의 타겟 구성: Counter({2.0: 37}) 



#### Stratified K-fold 분할

In [78]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=4)
# 분할 시 y를 고려해야 하기 때문에 split에 y를 입력!
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(f"Sample {i} ==> train_index: {train_index[:3]}, test_index: {test_index[:3]}")
    X_train, X_test, y_train, y_test = X.iloc[train_index, :], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    print("\tX_train의 비율: %0.2f, X_test의 비율: %0.2f" %(len(X_train)/len(X), len(X_test)/len(X)))
    print("\ty_train의 타겟 구성:", Counter(y_train['target']))
    print("\ty_test의 타겟 구성:", Counter(y_test['target']), "\n")

Sample 0 ==> train_index: [13 14 15], test_index: [0 1 2]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({1.0: 38, 0.0: 37, 2.0: 37})
	y_test의 타겟 구성: Counter({0.0: 13, 2.0: 13, 1.0: 12}) 

Sample 1 ==> train_index: [0 1 2], test_index: [13 14 15]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({1.0: 38, 0.0: 37, 2.0: 37})
	y_test의 타겟 구성: Counter({0.0: 13, 2.0: 13, 1.0: 12}) 

Sample 2 ==> train_index: [0 1 2], test_index: [26 27 28]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({0.0: 38, 2.0: 38, 1.0: 37})
	y_test의 타겟 구성: Counter({1.0: 13, 0.0: 12, 2.0: 12}) 

Sample 3 ==> train_index: [0 1 2], test_index: [38 39 40]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	y_train의 타겟 구성: Counter({0.0: 38, 2.0: 38, 1.0: 37})
	y_test의 타겟 구성: Counter({1.0: 13, 0.0: 12, 2.0: 12}) 



#### Group K-fold 분할
범주형 범수인 Group을 각 분할마다 검증용 데이터로 사용하는 K-fold 진행방식 (ex. G1 그룹 데이터들을 1번째 샘플의 Test set으로 활용)

In [68]:
# group 변수가 있는 데이터를 생성
## group의 수준은 g0, g1, g2, g3의 4종류가 있다.
iris2 = iris.copy()
iris2['group'] = iris2['target'].apply(lambda x: f"g{int(np.random.randint(0,4,1))}")
print(Counter(iris2['group']))
print(iris2.head(3))

Counter({'g1': 42, 'g3': 37, 'g0': 36, 'g2': 35})
   sepal length  sepal width  petal length  petal width  target group
0           5.1          3.5           1.4          0.2     0.0    g1
1           4.9          3.0           1.4          0.2     0.0    g2
2           4.7          3.2           1.3          0.2     0.0    g2


C:\Users\sunwa\AppData\Local\Temp\ipykernel_16420\862415442.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  iris2['group'] = iris2['target'].apply(lambda x: f"g{int(np.random.randint(0,4,1))}")


In [69]:
from sklearn.model_selection import GroupKFold
X = iris2.drop(['target', 'group'], axis=1)
y = iris2.filter(['target'])
group = iris2.filter(['group'])
gkf = GroupKFold(n_splits=4)
# 분할 시 group을 고려해야 하기 때문에 split에 group를 입력!
for i, (train_index, test_index) in enumerate(gkf.split(X, y, group)):
    print(f"Sample {i} ==> train_index: {train_index[:3]}, test_index: {test_index[:3]}")
    X_train, X_test, y_train, y_test = X.iloc[train_index, :], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    print("\tX_train의 비율: %0.2f, X_test의 비율: %0.2f" %(len(X_train)/len(X), len(X_test)/len(X)))
    print("\ttrain의 타겟 구성:", Counter(y_train['target']))
    print("\ttest의 타겟 구성:", Counter(y_test['target']))
    print("\ttrain의 그룹 구성:", Counter(group.iloc[train_index]['group']))
    print("\ttest의 그룹 구성:", Counter(group.iloc[test_index]['group']), "\n")

Sample 0 ==> train_index: [1 2 3], test_index: [0 5 6]
	X_train의 비율: 0.72, X_test의 비율: 0.28
	train의 타겟 구성: Counter({0.0: 40, 1.0: 34, 2.0: 34})
	test의 타겟 구성: Counter({1.0: 16, 2.0: 16, 0.0: 10})
	train의 그룹 구성: Counter({'g3': 37, 'g0': 36, 'g2': 35})
	test의 그룹 구성: Counter({'g1': 42}) 

Sample 1 ==> train_index: [0 1 2], test_index: [ 8 12 15]
	X_train의 비율: 0.75, X_test의 비율: 0.25
	train의 타겟 구성: Counter({2.0: 40, 0.0: 38, 1.0: 35})
	test의 타겟 구성: Counter({1.0: 15, 0.0: 12, 2.0: 10})
	train의 그룹 구성: Counter({'g1': 42, 'g0': 36, 'g2': 35})
	test의 그룹 구성: Counter({'g3': 37}) 

Sample 2 ==> train_index: [0 1 2], test_index: [ 4  7 11]
	X_train의 비율: 0.76, X_test의 비율: 0.24
	train의 타겟 구성: Counter({2.0: 39, 1.0: 38, 0.0: 37})
	test의 타겟 구성: Counter({0.0: 13, 1.0: 12, 2.0: 11})
	train의 그룹 구성: Counter({'g1': 42, 'g3': 37, 'g2': 35})
	test의 그룹 구성: Counter({'g0': 36}) 

Sample 3 ==> train_index: [0 4 5], test_index: [1 2 3]
	X_train의 비율: 0.77, X_test의 비율: 0.23
	train의 타겟 구성: Counter({1.0: 43, 2.0: 37, 0.

### **3-3. 교차 검증**

#### 분할 샘플들로 교차 검증

In [79]:
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.linear_model import LogisticRegression
X = iris.drop('target', axis=1)
y = iris['target']
LOGREG = LogisticRegression(max_iter = 300, C = 0.1) # 학습 모델 정의
SKF = StratifiedKFold(n_splits=4) # 데이터 분할 방법 정의
result = cross_validate(LOGREG, X, y, cv = SKF, return_train_score=True) #교차 검증
print(DataFrame(result))

   fit_time  score_time  test_score  train_score
0  0.034606    0.001000    0.894737     0.955357
1  0.005505    0.001000    0.947368     0.964286
2  0.005000    0.001002    0.945946     0.955752
3  0.004539    0.001018    1.000000     0.938053


#### 파라미터 후보들로 교차 검증

In [81]:
from sklearn.model_selection import GridSearchCV
LOGREG = LogisticRegression(max_iter = 300) # 학습 모델 정의
param_grid={'C':[0.01, 0.1, 1], 'solver':['lbfgs', 'liblinear']} #파라미터 후보 정의
SKF = StratifiedKFold(n_splits=4) # 데이터 분할 방법 정의
grid = GridSearchCV(LOGREG, param_grid, cv = SKF) #교차 검증
grid.fit(X, y)
print("최상의 교차 검증 점수: {:.5f}".format(grid.best_score_))
print("최적의 매개변수: {}".format(grid.best_params_))
print(DataFrame(grid.cv_results_))

최상의 교차 검증 점수: 0.97333
최적의 매개변수: {'C': 1, 'solver': 'lbfgs'}
   mean_fit_time  std_fit_time  mean_score_time  std_score_time  param_C  \
0       0.004003  7.077455e-04         0.001013    7.100861e-04     0.01   
1       0.001250  4.323286e-04         0.000751    4.334305e-04     0.01   
2       0.005782  8.424951e-04         0.000749    4.323620e-04     0.10   
3       0.001517  4.837355e-04         0.000462    4.652702e-04     0.10   
4       0.006763  5.498501e-04         0.001015    2.512150e-05     1.00   
5       0.001000  5.591411e-07         0.001000    2.665601e-07     1.00   

  param_solver                              params  split0_test_score  \
0        lbfgs      {'C': 0.01, 'solver': 'lbfgs'}           0.815789   
1    liblinear  {'C': 0.01, 'solver': 'liblinear'}           0.684211   
2        lbfgs       {'C': 0.1, 'solver': 'lbfgs'}           0.894737   
3    liblinear   {'C': 0.1, 'solver': 'liblinear'}           0.815789   
4        lbfgs         {'C': 1, 'solver': 

#### 연습문제 #1

In [82]:
from pandas import read_csv
df = read_csv('https://raw.githubusercontent.com/algoboni/pythoncodebook1-1/main/practice1_bank.csv')
print(df.head(3))

   age         job  marital  education default  balance housing loan  \
0   30  unemployed  married    primary      no     1787      no   no   
1   33    services  married  secondary      no     4789     yes  yes   
2   35  management   single   tertiary      no     1350     yes   no   

    contact month   y  
0  cellular   oct  no  
1  cellular   may  no  
2  cellular   apr  no  


In [98]:
# 범주형 변수들을 레이블 인코딩을 통해 전처리 한다.
df2 = df.copy()
for col in [i for i in df.columns if df2[i].dtypes==object]:
    DICT = dict(zip(df2[col].unique(), range(df2[col].nunique())))
    #DICT = dict(zip(df2[col].unique(), [i for i in range(df2[col].nunique())]))
    df2[col] = df2[col].map(DICT)
print(df.head(3))

# 마지막 변수의 DICT 확인
print("\nDICT 생성 확인")
print(df[col].unique())
print([i for i in range(df[col].nunique())])
print(DICT)

print(df2.head(3))

   age         job  marital  education default  balance housing loan  \
0   30  unemployed  married    primary      no     1787      no   no   
1   33    services  married  secondary      no     4789     yes  yes   
2   35  management   single   tertiary      no     1350     yes   no   

    contact month   y  
0  cellular   oct  no  
1  cellular   may  no  
2  cellular   apr  no  

DICT 생성 확인
['no' 'yes']
[0, 1]
{'no': 0, 'yes': 1}
   age  job  marital  education  default  balance  housing  loan  contact  \
0   30    0        0          0        0     1787        0     0        0   
1   33    1        0          1        0     4789        1     1        0   
2   35    2        1          2        0     1350        1     0        0   

   month  y  
0      0  0  
1      1  0  
2      2  0  


In [92]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier() # 학습 모델 정의
param_grid={'max_depth':[5, 6, 7, 8]} #파라미터 후보 정의
SKF = StratifiedKFold(n_splits=6) # 데이터 분할 방법 정의
grid = GridSearchCV(rf, param_grid, cv = SKF) #교차 검증

X = df2.drop('y', axis=1)
y = df2['y']
grid.fit(X, y)

print("최상의 교차 검증 점수: {:.2f}".format(grid.best_score_))
print("최적의 매개변수: {}".format(grid.best_params_))
print(DataFrame(grid.cv_results_))
# 매개변수들을 활용한 교차분석 결과, max_depth가 7일 때 가장 높은 성능을 보이는 것으로 확인하였다.

최상의 교차 검증 점수: 0.88
최적의 매개변수: {'max_depth': 7}
   mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.157216      0.022431         0.003956        0.000688   
1       0.137484      0.001359         0.004351        0.001498   
2       0.150318      0.004811         0.003681        0.001714   
3       0.160886      0.001128         0.004721        0.000480   

   param_max_depth            params  split0_test_score  split1_test_score  \
0                5  {'max_depth': 5}           0.885942           0.884615   
1                6  {'max_depth': 6}           0.887268           0.883289   
2                7  {'max_depth': 7}           0.885942           0.885942   
3                8  {'max_depth': 8}           0.885942           0.883289   

   split2_test_score  split3_test_score  split4_test_score  split5_test_score  \
0           0.881963           0.885790           0.883134           0.884462   
1           0.881963           0.885790           0.883134       